# Follow up from last time
It seemed like this one was "easy".

A few general comments:

* DROP USER IF EXISTS <br>
    When to use it - always, or is it a sign of problems?
* Users missing in the backup

* 3-2-1 backup (three copies, on two media, 1 off premise)

# Stored procedures, json

### Learning goals

After this week, you should be able to:

* Decide when to use stored procedures in your architecture
* Be able to write stored procedures for MySQL
* Be able to call stored procedures from client applications
* Set up the permission system of MySQL so that the client applications can only modify through stored procedures
* Set up triggers in MySQL that call stored procedures
* Set up views and materialized views
* Use simple MySQL JSON operators
* Further investigate the json and document storage of MySQL


# Exercises (full text at the assignment page)

Stackexhange data:
https://archive.org/details/stackexchange.

The programming stackexchange (stackoverflow) is about 40GB compressed XML - I guestimate around 100GB expanded.

I suggest you use the two dataserts for "coffee.stackexchange.com" (small - for debugging purposes), and "askubuntu.com" which is around 700MB compressed - expands into 2GB XML.

### Exercise 1
Write a stored procedure `denormalizeComments(postID)` that moves all comments to a post (the parameter) into a json array.  

### Exercise 2
Create a trigger such that new adding new comments to a post triggers an insertion of that comment in the json array from exercise 1.

### Exercise 3
Rather than using a trigger, create a stored procedure to add a comment to a post - adding it both to the comment table and the json array

### Exercise 4
Make a materialized view that has json objects with questions and its answeres, but no comments. Both the question and each of the answers must have the display name of the user, the text body, and the score.

### Exercise 5
Using the materialized view, create a stored procedure with one parameter `keyword`, which returns all posts where the keyword appears at least once, and where at least two comments mention the keyword as well.


In [ ]:
!docker container ls -a

In [ ]:
%%bash
docker run \
--rm \
--name my_mysql \
-v $(pwd)/mysql_databasefiles:/var/lib/mysql \
-p 3306:3306 \
-e MYSQL_ROOT_PASSWORD=deterentysker!42snapsnap \
-d \
mysql
echo "MySQLRunning"

In [ ]:
import sys
import mysql.connector

def rootconnect():
    try:
        pw = 'deterentysker!42snapsnap'
        conn = mysql.connector.connect( host='localhost', database='classicmodels',user='root', password=pw)
        conn.autocommit = True
        return conn;
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    

conn = rootconnect()

def sqlQuery(sqlString):
    global conn
    try:
        if not conn.is_connected():
            conn = rootconnect()
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchall()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

def sqlDo(sqlString):
    global conn
    try:
        if not conn.is_connected():
            conn = rootconnect()
        cursor = conn.cursor()
        cursor.execute(sqlString)
        res = cursor.fetchwarnings()
        return res
    except Exception as ex:
        print(str(ex), file=sys.stderr)
    finally:    
        cursor.close()

"Done"    

## What is a stored procedure

![](http://www.mysqltutorial.org/wp-content/uploads/2009/12/mysql-stored-procedure1.jpg)

* The stored procedures are still connected to the database and is not running inside the kernel
* Stored procedures are semi-standarized
* What is the run-time of a stored procedure (stack, lifetime of global variable, connection to schemas)

### Advantages

* Stored procedures help **increase the performance** of the applications. However, MySQL stored procedures are compiled and cached (per connection).
* Reduce the traffic between application and database server.
* Reusable and transparent to any applications. 
* Stored procedures expose the database interface to all applications.
* Stored procedures are secure. 

### Disadvanteges
* If you use many stored procedures, the memory usage of every connection that is using those stored procedures will increase substantially. 
* Stored procedure’s constructs are not designed for developing complex and flexible business logic.
* It is difficult to debug stored procedures. 
* It is not easy to develop and maintain stored procedures.

## Some really simple SQL

In [ ]:
sqlQuery("Select 1, 2+4, 'Cat emojoi: 🐈';")

In [ ]:
sqlQuery("select curdate(), adddate(curdate(),7)")

# Stored procedures

### Classic model database
![](images/ClassicModelsER.png)

# Which office has the most customers
```sql
select city, count(customerNumber)
from offices, employees, customers
where employee.officeCode = offices.officeCode and ....
```

In [ ]:
sqlQuery("call mostcustomers();")

In [ ]:
sqlDo("""
CREATE PROCEDURE mostcustomers()
BEGIN
  SELECT 'Lyngby', 9000;
END;
""")

In [ ]:
sqlQuery("call mostcustomers();")

#### Check up in workbench - there is now a stored procedure

In [ ]:
sqlDo("DROP PROCEDURE IF EXISTS `classicmodels`.`mostcustomers`;")

In [ ]:
sqlDo("""
CREATE PROCEDURE classicmodels.mostcustomers()
BEGIN
    select offices.city, count(customerNumber) as customers
    from offices, employees, customers
    where employees.officeCode = offices.officeCode and customers.salesRepEmployeeNumber = employees.employeeNumber
    group by offices.city
    order by customers desc
    limit 5;
END;
""")

In [ ]:
sqlQuery("call mostcustomers();")

## Parameters to stored procedures

In [ ]:
sqlQuery("select productline from productlines")

In [ ]:
sqlQuery("call products('Ships')")

In [ ]:
sqlDo("""
CREATE PROCEDURE classicmodels.products(IN p_line VARCHAR(255))
BEGIN
    select productCode, productName
    from products
    where productLine = p_line;
END
""")

In [ ]:
sqlQuery("call products('Ships')")

# Your turn

Create a stored procedure which takes a city of office as parameter, and returns how many orders each employee have for that office.

You can make your solution using workbench or any other tool. 

### Lets see some solutions
[Our scrapbook](https://docs.google.com/document/d/1c8W921VuAG5FaQJuMjIn6JSdAuYiA2eBQxrKk-YYXVs/edit) 

## More on stored procedures

We saw an IN parameter. There is also an OUT parameter. 

In larger stored procedure code, there are global variables as well as a lot of other stuff - it is full programming languages.

#### DELIMITER
This is a special command used in the shell of mysql

Normally each statement is ended by ";". 

The mysql shell reads uptil then next ";", and fires that as a statement to the mysql engine.

The delimiter command changes the delimiter from ";" to something else.

```sql
DELIMITER //
CREATE PROCEDURE GetOfficeByCountry(IN countryName VARCHAR(255))
 BEGIN
 SELECT * 
 FROM offices
 WHERE country = countryName;
 END //
DELIMITER ;
```

### But the language is so ugly

Different databases use different languages for stored procedures.

You might see C#, Java, Python run in the database server.

[Our friend stackoverflow has a good answer - but it is getting old](https://stackoverflow.com/questions/339744/better-languages-than-sql-for-stored-procedures)


# Triggers
A trigger is a piece of code which is executed when an event happens.

A trigger or database trigger is a stored program executed automatically to respond to a specific event e.g.,  **insert, update or delete** occurred in a table.

(This section on triggers is inspired by [MySQLTutorial.org](http://www.mysqltutorial.org/mysql-triggers.aspx))

### Creating a log of changes
![](images/EmployeesTableTrigger.png)

### Logging table:

```sql
CREATE TABLE employees_audit (
    id INT AUTO_INCREMENT PRIMARY KEY,
    employeeNumber INT NOT NULL,
    lastname VARCHAR(50) NOT NULL,
    changedat DATETIME DEFAULT NULL,
    action VARCHAR(50) DEFAULT NULL
);
```

### The trigger
```sql
DELIMITER $$
CREATE TRIGGER before_employee_update 
    BEFORE UPDATE ON employees
    FOR EACH ROW 
BEGIN
    INSERT INTO employees_audit
    SET action = 'update',
        employeeNumber = OLD.employeeNumber,
        lastname = OLD.lastname,
        changedat = NOW(); 
END$$
DELIMITER ;
```

# Triggers - advantages

* SQL triggers provide an alternative way to check the integrity of data.
* SQL triggers can catch errors in business logic in the database layer.
* SQL triggers provide an alternative way to run scheduled tasks. By using SQL triggers, you don’t have to wait to run the scheduled tasks because the triggers are invoked automatically before or after a change is made to the data in the tables.
* SQL triggers are very useful to audit the changes of data in tables.

# Triggers - disadvantages

* SQL triggers only can provide an extended validation and they cannot replace all the validations. Some simple validations have to be done in the application layer. For example, you can validate user’s inputs in the client side by using JavaScript or on the server side using server-side scripting languages such as JSP, PHP, ASP.NET, Perl.
* SQL triggers are invoked and executed invisible from the client applications, therefore, it is difficult to figure out what happens in the database layer.
* SQL triggers may increase the overhead of the database server.

### Stored procedure or trigger

If an action requires a trigger - consider making a stored procedure which do both actions:
* the trigger action
* the original action

### Timed triggers

```sql
CREATE EVENT every_minute_for_an_hour_starting_now
ON SCHEDULE EVERY 1 MINUTE
STARTS CURRENT_TIMESTAMP
ENDS CURRENT_TIMESTAMP + INTERVAL 1 HOUR
DO
   INSERT INTO messages(message,created_at)
   VALUES('Test MySQL recurring Event',NOW());
```

# Views

Views are basically a named select statement.

![](http://www.mysqltutorial.org/wp-content/uploads/2009/12/mysql-view.jpg)

```sql
CREATE VIEW view_name [(column_list)] AS
select-statement;
```

In [ ]:
sqlDo("""
DROP VIEW IF EXISTS customer_admin;
CREATE VIEW customer_admin AS
select customerNumber, customerName, customers.phone, 
    concat(firstName,' ',lastName) as employeeName, offices.city
from customers
    inner join employees on customers.salesRepEmployeeNumber = employees.employeeNumber
    inner join offices on employees.officeCode = offices.officeCode
""")

In [ ]:
sqlQuery("""
select * from customer_admin 
limit 5
""")

## Materialized view

A materialized view is a view which is cashed in the database. 

MySQL do not have such a thing.

To implement it you need to do:

* Create a table to store the result
* Create a stored procedure which will update it based on the (non-material) view
* Perhaps make a trigger that updates the material view on demand or on a timer

# Your turn

Create a view `customer_balance` which for each customer has customerNumber, customerName, sum of all payments by the customer and sum of all price of all orders.

Start with the sum of all payments, as the sum of all orders are a bit harder.

You can make the "sum of all payments" by creating intermediate views.
* A view of orderdetails which has the price calculated
* A view of orders which has the total price calculated

### Lets see some solutions
[Our scrapbook](https://docs.google.com/document/d/1c8W921VuAG5FaQJuMjIn6JSdAuYiA2eBQxrKk-YYXVs/edit) 

# JSON support

In MySQL JSON is supported as a spcial column type:


In [ ]:
sqlQuery("SELECT JSON_ARRAY('a', 1, NOW())")

In [ ]:
sqlQuery("SELECT JSON_OBJECT('key1', 1, 'key2', 'abc');")

### JSON from existing data

![](images/ClassicModelsProductLines.png)

In [ ]:
#Collecting into JSON arrays

sqlQuery("""
select productLine, JSON_ARRAYAGG(productName) 
from products
group by productLine
limit 2
""")

### Making orderdetails a json array of objects

![](images/ClassicModelsOrders.png)

In [ ]:
sqlQuery("""
select orderNumber, JSON_OBJECT('prodCode', productCode, 'qty', 
        quantityOrdered, 'price', priceEach, 'line', orderLineNumber) as detail
from orderdetails
limit 4
""")

Notice:
* I keep the orderNumber foreign key 
* There is no order to the attributes in the JSON objects
* The JSON_OBJECT function take pairs of arguments

### Subqueries and join and json

In [ ]:
sqlQuery("""
select orders.orderNumber, JSON_ARRAYAGG(detail) as details
from orders 
inner join (select orderNumber, JSON_OBJECT('line', orderLineNumber, 'prodCode', productCode, 'qty', 
                quantityOrdered, 'price', priceEach, 'total', quantityOrdered*priceEach) as detail
            from orderdetails) as detail_table
    on orders.orderNumber = detail_table.orderNumber
group by orders.orderNumber
limit 2
""")

In [ ]:
sqlDo("""
DROP VIEW IF EXISTS order_details;
CREATE VIEW order_details AS
select orders.orderNumber, JSON_ARRAYAGG(detail) as details
from orders 
inner join (select orderNumber, JSON_OBJECT('line', orderLineNumber, 'prodCode', productCode, 'qty', 
                quantityOrdered, 'price', priceEach, 'total', quantityOrdered*priceEach) as detail
            from orderdetails) as detail_table
    on orders.orderNumber = detail_table.orderNumber
group by orders.orderNumber
""")

In [ ]:
sqlQuery("select * from order_details limit 3")

# Your Turn - Assignment 5